In [1]:
import h5py
import numpy as np

In [32]:
with h5py.File("mytestfile.hdf5", "w") as f:
    dset = f.create_dataset("mydataset", (100,), dtype='i')
    print dset.shape, dset.dtype
    dset[...] = np.arange(100)
    print dset[0:100:10], '\n'
    print f.name, type(f), '\n'
    print dset.name, type(dset), '\n'
    grp = f.create_group("subgroup")
    print grp.name, type(grp), '\n'
    dset2 = grp.create_dataset("another_dataset", (50,), dtype='f')
    print dset2.name, type(dset2), '\n'
    dset3 = f.create_dataset('subgroup2/dataset_three', (10,), dtype='i')
    print dset3.name, type(dset3), '\n'
    # Attributes of the dataset
    dset.attrs['temperature'] = 99.5

(100,) int32
[ 0 10 20 30 40 50 60 70 80 90] 

/ <class 'h5py._hl.files.File'> 

/mydataset <class 'h5py._hl.dataset.Dataset'> 

/subgroup <class 'h5py._hl.group.Group'> 

/subgroup/another_dataset <class 'h5py._hl.dataset.Dataset'> 

/subgroup2/dataset_three <class 'h5py._hl.dataset.Dataset'> 



In [39]:
# h5py with pandas
from pandas import HDFStore, DataFrame

with HDFStore('storage.h5') as hdf:
    df = DataFrame(np.random.rand(5,3), columns=('A','B','C'))
    hdf.put('d1', df, format='table', data_columns=True)
    print hdf['d1'].shape
    hdf.append('d1', DataFrame(np.random.rand(5,3), columns=('A','B','C')), 
               format='table', data_columns=True)
    print type(hdf['d1'])

(5, 3)
<class 'pandas.core.frame.DataFrame'>


In [42]:
from pandas import read_hdf

hdf = read_hdf('storage.h5', 'd1', where='A>.5', columns=('A', 'B'))
print hdf

          A         B
2  0.537706  0.771100
3  0.926372  0.729073
1  0.587668  0.034781
3  0.621757  0.624549


In [47]:
with HDFStore('storage.h5') as hdf:
    hdf.put('tables/t1', DataFrame(np.random.rand(20, 5)))
    hdf.put('tables/t2', DataFrame(np.random.rand(10,3)))
    hdf.put('new_tables/t1', DataFrame(np.random.rand(15,2)))
    print hdf

<class 'pandas.io.pytables.HDFStore'>
File path: storage.h5
/d1                       frame_table  (typ->appendable,nrows->10,ncols->3,indexers->[index],dc->[A,B,C])
/new_tables/t1            frame        (shape->[15,2])                                                   
/tables/t1                frame        (shape->[20,5])                                                   
/tables/t2                frame        (shape->[10,3])                                                   


In [56]:
from tables import *

class Particle(IsDescription):
    identity = StringCol(itemsize=22, dflt=" ", pos=0)  # character String
    idnumber = Int16Col(dflt=1, pos = 1)  # short integer
    speed    = Float32Col(dflt=1, pos = 2)  # single-precision

fileh = open_file('objecttree.h5', mode='w')

root = fileh.root
print 'Root: ', root

group1 = fileh.create_group(root, "group1")
group2 = fileh.create_group(root, "group2")

array1 = fileh.create_array(root, "array1", ["string", "array"], "String array")

table1 = fileh.create_table(group1, "table1", Particle)
table2 = fileh.create_table("/group2", "table2", Particle)
array2 = fileh.create_array("/group1", "array2", [1,2,3,4])

print fileh

# Now, fill the tables
for table in (table1, table2):
    # Get the record object associated with the table:
    row = table.row

    # Fill the table with 10 records
    for i in xrange(10):
        # First, assign the values to the Particle record
        row['identity']  = 'This is particle: %2d' % (i)
        row['idnumber'] = i
        row['speed']  = i * 2.

        # This injects the Record values
        row.append()

    # Flush the table buffers
    table.flush()
    
print fileh

fileh.close()

Root:  / (RootGroup) u''
objecttree.h5 (File) u''
Last modif.: 'Fri May 27 13:54:07 2016'
Object Tree: 
/ (RootGroup) u''
/array1 (Array(2,)) 'String array'
/group1 (Group) u''
/group1/array2 (Array(4,)) ''
/group1/table1 (Table(0,)) ''
/group2 (Group) u''
/group2/table2 (Table(0,)) ''

objecttree.h5 (File) u''
Last modif.: 'Fri May 27 13:54:07 2016'
Object Tree: 
/ (RootGroup) u''
/array1 (Array(2,)) 'String array'
/group1 (Group) u''
/group1/array2 (Array(4,)) ''
/group1/table1 (Table(10,)) ''
/group2 (Group) u''
/group2/table2 (Table(10,)) ''

